# 1: QUBOの定義とnumpy行列による表現
二次二値関数最適化(Quadratic Unconstrained Binary Optimization; QUBO)問題とは次のように定義される最小化問題である。

$$ \min_{\mathbf x\in\{0,1\}^N}\sum_{i\leq j}q_{ij}x_ix_j.$$

ここで$N$は問題の次元数、上三角行列$Q=\{q_{ij}\in\mathbb R\}$は問題を定義するパラメータである。

このQUBO問題自体はQをnumpy行列として作成すれば表現可能である。
例として、

$$
\begin{align}
f(x) &= x_1-1x_2-2x_3 \\
 & +2x_1x_2-x_1x_3 \\
 & -2x_2x_3-x_2x_4 \\
 & +2x_3x_4
\end{align}
$$
という問題を考える。4ビットの問題なので解の候補は$2^4=16$通り。
先に解を全通り試しておくと

|$x_1$|$x_2$|$x_3$|$x_4$| f(x) |
|:-:|:-:|:-:|:-:|:-:|
|0|0|0|0|0|
|0|0|0|1|0|
|0|0|1|0|-2|
|0|0|1|1|0|
|0|1|0|0|-1|
|0|1|0|1|-2|
|0|1|1|0|-5|
|0|1|1|1|-4|
|1|0|0|0|1|
|1|0|0|1|1|
|1|0|1|0|-2|
|1|0|1|1|0|
|1|1|0|0|2|
|1|1|0|1|1|
|1|1|1|0|-3|
|1|1|1|1|-2|

で、最適解は$\mathbf x= [0,1,1,0]^\top$であると分かる。この問題の行列表現は
$$
Q=\begin{pmatrix}
1 & 2 & -1 & 0 \\
0 & -1 & -2 & -1 \\
0 & 0 & -2 & 2 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}
$$
なので、numpy行列としてこれを作成する。

In [1]:
import numpy as np
Q = np.array([
    [1, 2, -1, 0],
    [0, -1, -2, -1],
    [0, 0, -2, 2],
    [0, 0, 0, 0]
])

def xQx(x, Q):
    return x.dot(Q.dot(x))

assert xQx(np.array([0,1,1,1]), Q) == -4

これを解くために、ここではD-Wave Systems社がPythonクラスとして提供しているQUBO問題のソルバーを利用する。ひとまず4種類挙げておく。

| クラス名 | アルゴリズム |
|:--|:--|
|dwave_qbsolv.QBSolv| タブー探索 |
|neal.SimulatedAnnealingSampler | 焼きなまし法(SA) |
|dwave.system.samplers.DWaveSampler | 量子アニーリング(QA) |
|dwave.system.composites.AutoEmbeddingComposite|量子アニーリング(QA)|

QAのソルバーが二つある理由は後述する。

## 1.1: タブー探索による解法

まずはタブー探索で解いてみる。ソルバーのクラスのインスタンスを作成したら、その`sample_qubo`メソッドに先ほど作成した`Q`を与えればQUBO問題を解いてくれる。

In [2]:
# pip install dwave_qbsolv でインストールしておく
from dwave_qbsolv import QBSolv
qbsolv = QBSolv()

res = qbsolv.sample_qubo(Q)
res

SampleSet(rec.array([([0, 1, 1, 0], -5., 51)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), [0, 1, 2, 3], {}, 'BINARY')

この出力から、答えは`dimod.sampleset.SampleSet`クラスのオブジェクトとして返されており、
その中身は`rec.array`というオブジェクトと\[0,1,2,3\]というリスト、空の辞書\{\}と'BINARY'という文字列を含んでいるらしい。その内容は[GitHub上のソースコード](https://github.com/dwavesystems/dimod/blob/126a9056c4226d6955395620efb94ebee07581a5/dimod/sampleset.py#L311)を見ることで分かるが、`record`というメンバ変数にアクセスすることで取得できる。

In [3]:
res.record

rec.array([([0, 1, 1, 0], -5., 51)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')])

これはnumpyのrec.arrayというクラス(record arrayの略)のオブジェクトで、データの本体である\[(\[0, 1, 1, 0\], -5., 51)\]というリストのそれぞれの要素が'sample'と呼ばれる長さ4の整数(integer)、'energy'と呼ばれる浮動小数点数、'num_occurrences'と呼ばれる整数からなることを表している。

つまりQBSolvを実行した結果、$x=[0, 1, 1, 0]^\top$というエネルギー-5の解が51回出現したということである。
rec.arrayオブジェクトは名前を添字としてそれぞれの配列にアクセスできる。

In [4]:
print(res.record['sample'])
print(res.record['energy'])
print(res.record['num_occurrences'])

[[0 1 1 0]]
[-5.]
[51]


また、その他の情報も`variables`, `info`, `vartype`というメンバ変数(プロパティ)にアクセスすると得られる。

In [5]:
res.variables # 変数名を表す。今回は行列の形であたえたので、インデックスである0〜3がそのまま並んでいる。

Variables([0, 1, 2, 3])

In [6]:
res.info # 解いたときの付加的な情報が格納される。QBSolvは特に情報を残さないので空っぽ。

{}

In [7]:
res.vartype # QUBO問題は変数が0/1なのでBINARYと表示される。変数が-1/+1であるSPINもある。

<Vartype.BINARY: frozenset({0, 1})>

例えば、変数名を0, 1, 2, 3ではなく a, b, c, dなどとしたい場合は、Qを辞書オブジェクトで次のように表現する。

In [8]:
Q = {
 ('a', 'a'): 1,
 ('a', 'b'): 2,
 ('a', 'c'): -1,
 ('b', 'b'): -1,
 ('b', 'c'): -2,
 ('b', 'd'): -1,
 ('c', 'c'): -2,
 ('c', 'd'): 2
}
res = qbsolv.sample_qubo(Q)
res.variables

Variables(['a', 'b', 'c', 'd'])

タブー探索のアルゴリズム自体には全く触れなかったが、詳細は[\[Glover　& Laguna, 1998\]](https://link.springer.com/chapter/10.1007%2F978-1-4613-0303-9_33)等を参照。

## 1.2: 焼きなまし法による解法
焼きなまし法でもまったく同様に解くことができる。

In [9]:
# pip install dwave_neal でインストールしておく。
from neal import SimulatedAnnealingSampler
sa_sampler = SimulatedAnnealingSampler()

res = sa_sampler.sample_qubo(Q)
res

SampleSet(rec.array([([0, 1, 1, 0], -5., 1)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['a', 'b', 'c', 'd'], {'beta_range': [0.2772588722239781, 18.420680743952367], 'beta_schedule_type': 'geometric'}, 'BINARY')

ちなみにSamplerという名称や`sample_qubo`というメソッドの名称は、これらの解法がヒューリスティクスなので最適解が得られることを保証しておらず、あくまで得られるのは確率的に選ばれてくる(つまりサンプリングされてくる)解であることを意味している。

焼きなまし法では`info`に付加情報が入っている。

In [10]:
res.info

{'beta_range': [0.2772588722239781, 18.420680743952367],
 'beta_schedule_type': 'geometric'}

これは焼きなまし法における逆温度$\beta$のスケジューリングに関するハイパーパラメータである。詳細は省略する。

## 1.3: 量子アニーリングによる解法
D-Waveの量子アニーリングマシンの実機をつかって解をサンプリングするには`DWaveSampler`クラスを使う。
実機へのアクセスには認証用のトークンが必要なので、
変数`DWAVE_TOKEN`にあらかじめセットしておく。

In [11]:
DWAVE_TOKEN="" #"replace this string with actual token"

In [12]:
from dwave.system.samplers import DWaveSampler
dw_sampler = DWaveSampler(
    endpoint="https://cloud.dwavesys.com/sapi",
    solver = 'DW_2000Q_6',
    token = DWAVE_TOKEN
)

実はこれで問題を解こうとするとエラーが発生する。

In [13]:
dw_sampler.sample_qubo(Q) # ERROR

BinaryQuadraticModelStructureError: Problem graph incompatible with solver.

エラーの内容は`BinaryQuadraticModelStructureError: Problem graph incompatible with solver.`
つまり、D-Waveマシン上で量子ビットが実際には繋がっていないところにQの要素が設定されていることで生じているエラーである。

D-Wave 2000Qの結合パターンはキメラグラフという(非平面グラフではあるが)疎結合な形なので、解きたい問題をその形に限定するのは無理がある。
そこで埋め込みと呼ばれるマッピング処理で複数の量子ビットがひとつの論理ビットを表すようにグルーピングしていくと、Qが任意のビット間に要素をもっていてもD-Waveマシン上に対応する結合が存在するようになる。

D-Wave 2000Qの構造では、最大64ビットまでであれば任意のQを埋め込みによって解くことができる。(ただし、実機には壊れて動作しない量子ビットがあるので実際はもう少し小さいサイズまで。)この埋め込み処理を自動でおこなってくれるのが`dwave.system.composites.AutoEmbeddingComposite`クラスである。初期化時に内部で使う`DWaveSampler`を与えるとそれに合わせて勝手に埋め込みをしてくれる。

In [14]:
from dwave.system.composites import AutoEmbeddingComposite
qa_sampler = AutoEmbeddingComposite(dw_sampler)

res = qa_sampler.sample_qubo(Q, num_reads=10)
res

SampleSet(rec.array([([0, 1, 1, 0], -5., 10, 0.)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), ['a', 'b', 'c', 'd'], {'timing': {'qpu_sampling_time': 2389, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 198, 'qpu_access_time': 13131, 'qpu_access_overhead_time': 1092, 'qpu_programming_time': 10741, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 67, 'post_processing_overhead_time': 67, 'total_real_time': 13131, 'run_time_chip': 2389, 'anneal_time_per_run': 20, 'readout_time_per_run': 198}, 'problem_id': 'd4a62609-b7e5-4233-acc9-8f918652582d'}, 'BINARY')

num_readsはサンプリング回数を指定したいときに使う。`info`を見ると色々なハードウェア設定が格納されている。

In [15]:
res.info

{'timing': {'qpu_sampling_time': 2389,
  'qpu_anneal_time_per_sample': 20,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 13131,
  'qpu_access_overhead_time': 1092,
  'qpu_programming_time': 10741,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 67,
  'post_processing_overhead_time': 67,
  'total_real_time': 13131,
  'run_time_chip': 2389,
  'anneal_time_per_run': 20,
  'readout_time_per_run': 198},
 'problem_id': 'd4a62609-b7e5-4233-acc9-8f918652582d'}

この中で最も重要なのが`anneal_time_per_run`の項目で、サンプリング一回のアニーリング(ハミルトニアンの変化)に20μsをかけたことを表している。
サンプリングする際に`annealing_time`というオプションで設定することができる。

In [16]:
res = qa_sampler.sample_qubo(Q, annealing_time=50, num_reads=10)
res.info

{'timing': {'qpu_sampling_time': 2689,
  'qpu_anneal_time_per_sample': 50,
  'qpu_readout_time_per_sample': 198,
  'qpu_access_time': 13432,
  'qpu_access_overhead_time': 2375,
  'qpu_programming_time': 10743,
  'qpu_delay_time_per_sample': 21,
  'total_post_processing_time': 130,
  'post_processing_overhead_time': 130,
  'total_real_time': 13432,
  'run_time_chip': 2689,
  'anneal_time_per_run': 50,
  'readout_time_per_run': 198},
 'problem_id': 'c2c8966e-9077-4dd4-bf3c-23ef603625c5'}

そのほかにQAを呼び出すときに使いそうな主なオプションをリストにしておく。詳細は[公式のマニュアルを参照](https://docs.dwavesys.com/docs/latest/c_solver_1.html)。
また、マシンの詳細スペックはDWaveSamplerクラスのオブジェクトの`properties`にも格納されているので必要に応じて適宜チェックするとよい。

|オプション名| 役割|値域 | デフォルト値 | リンク |
|----|----|:--:|:--:|:--:|
| num_reads | サンプリングを行う回数 |1~1000の整数| 1|[リンク](https://docs.dwavesys.com/docs/latest/c_solver_1.html#num-reads) |
|auto_scale | h, Jの自動スケーリングをおこなうかどうか | 真偽値 | True | [リンク](https://docs.dwavesys.com/docs/latest/c_solver_1.html#auto-scale) |
| annealing_time | 1回のアニーリング実行に使用する時間 [μs] | 1~2000 | 20 | [リンク](https://docs.dwavesys.com/docs/latest/c_solver_1.html#annealing-time) |
| anneal_schedule | アニーリングスケジュール(0→1)の動かし方を指定  (annealing_timeとは排他利用) | タプル$$(x,y)\in[0,2000]\otimes[0.0,1.0]$$のリスト | なし | [リンク](https://docs.dwavesys.com/docs/latest/c_solver_1.html#anneal-sdhedule) |

# 2: BQMオブジェクトの作成
各種サンプラーは`sample_qubo`だけでなく`sample_ising`というメソッドも持っていて、　QUBO問題だけでなく次のイジングモデルからのサンプリングも実行可能である。

$$H(\sigma\in\{-1,1\}^N)  = \sum_{i=1}^N h_i\sigma_i + \sum_{i<j}^N J_{ij}\sigma_i\sigma_j $$

これを最小化するような$\sigma$が確率的にサンプリングされてくるはずであるが、これはQUBOと完全に等価な問題である。D-Wave Systems社のライブラリでは、`dimod.BQM`というクラスで両方の表現を統一的に扱う仕組みを提供している。たとえば上記のQから`BQM`クラスのインスタンスを作成してみる。

In [17]:
from dimod import BQM
bqm = BQM.from_qubo(Q)
bqm

BinaryQuadraticModel({a: 1.0, b: -1.0, c: -2.0, d: 0.0}, {('a', 'b'): 2, ('a', 'c'): -1, ('b', 'c'): -2, ('b', 'd'): -1, ('c', 'd'): 2}, 0, 'BINARY')

`energy`や`energies`メソッドを使って目的関数の値を計算することが可能。

In [18]:
bqm.energy({'a': 0, 'b': 1, 'c': 1, 'd': 1})

-4.0

イジングモデルへの変換も容易。`change_vartype`メソッドで`'SPIN'`を指定する(逆方向の変換の場合は`'BINARY'`と指定)。

In [19]:
bqm_ising = bqm.change_vartype('SPIN', inplace=False)
bqm_ising

BinaryQuadraticModel({a: 0.75, b: -0.75, c: -1.25, d: 0.25}, {('a', 'b'): 0.5, ('a', 'c'): -0.25, ('b', 'c'): -0.5, ('b', 'd'): -0.25, ('c', 'd'): 0.5}, -1.0, 'SPIN')

In [20]:
bqm_ising.energy({'a': -1, 'b': 1, 'c': 1, 'd': 1})

-4.0

## 2.1: BQMのままサンプラーに投げる
BQMのインスタンスをサンプラーの`sample`メソッドに渡すとそのまま解いてくれる。

In [21]:
print(sa_sampler.sample(bqm).record) # 内部でsample_quboを呼び出してくれる
print(sa_sampler.sample(bqm_ising).record) # 内部でsample_isingを呼び出してくれる

[([0, 1, 1, 0], -5., 1)]
[([-1,  1,  1, -1], -5., 1)]


## 2.2: 制約の付加
QUBOを使っていくつかの種類の制約を表現することができる。例えば https://docs.dwavesys.com/docs/latest/c_gs_6.html の説明に乗っ取り、a, b, cのうちひとつのビットだけが1で他は0となるような制約は次のように表現できる。

$$ \min_{a,b,c} (a + b + c - 1)^2 = \min_{a,b,c}\left( [a,b,c]\cdot\begin{pmatrix}-1 & 2 & 2 \\ 0 & -1 & 2 \\ 0 & 0 & -1 \\ \end{pmatrix}\cdot[a,b,c]^\top \right)$$

In [22]:
choose1_constraint = BQM.from_qubo({
    ('a', 'a'): -1, ('b', 'b'): -1, ('c', 'c'): -1,
    ('a', 'b'): 2, ('a', 'c'): 2, ('b', 'c'): 2
})

SAを使ってこのQUBOから100回サンプリングしてみると、\[0,0,1\], \[0,1,0\], \[1,0,0\]がすべて現れることがわかる。
(SAは結果を集計せずに100回のサンプリング結果を生で表示するので、SampleSetを作り直してaggregateしてから表示させている。)

In [23]:
from dimod import SampleSet
SampleSet.from_samples_bqm(
    sa_sampler.sample(choose1_constraint, num_reads=100),
    choose1_constraint, aggregate_samples=True
)

SampleSet(rec.array([([0, 1, 0], -1., 30), ([1, 0, 0], -1., 48),
           ([0, 0, 1], -1., 22)],
          dtype=[('sample', 'i1', (3,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['a', 'b', 'c'], {}, 'BINARY')

この制約を表現するBQMをもとのQUBO問題を表すBQMに適当な重みで足し合わせると、制約を考慮しつつもとの問題を解くBQMができる。具体的には制約を破ることによって生じるペナルティが、元の問題のエネルギー差よりも大きければ十分。

ひとつの目安として、元の問題のエネルギーギャップが最小と最大の間で7なので重みを7.0にしてみる。
BQMには足し算や掛け算の処理が実装されていないので、`add_variable`および`add_interaction`メソッドを用いる。
実際にはこの値はわからないので、重みは制約が満たされるようになるまで少しずつ増やしていくのがよい。

In [24]:
bqm_constrained = bqm.copy()
W = 7# 制約条件の重み
for i, qii in choose1_constraint.iter_linear():
    bqm_constrained.add_variable(i, qii * W)
for i, j, qij in choose1_constraint.iter_quadratic():
    bqm_constrained.add_interaction(i,j, qij * W)

In [25]:
SampleSet.from_samples_bqm(
    sa_sampler.sample(bqm_constrained, num_reads=100),
    bqm, aggregate_samples=True
)

SampleSet(rec.array([([0, 0, 1, 0], -2., 49), ([0, 1, 0, 1], -2., 48),
           ([1, 0, 0, 0],  1.,  2), ([1, 0, 0, 1],  1.,  1)],
          dtype=[('sample', 'i1', (4,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['a', 'b', 'c', 'd'], {}, 'BINARY')

実際にサンプリングしてみると少数だが最適解以外の解も出てきていることがわかる。制約条件の重みを強くしすぎると、制約は満たすが最適解ではないものが多く出てくるようになるので注意。

## 2.3: PyQUBOの案内
問題と制約の合成は煩雑なので、制約を加味したQUBOを簡単に作成できるPyQUBOなどで問題を作成しておいてからこのBQMオブジェクトに変換すると手間が減るかもしれない。

参考:
* [PyQUBO](https://github.com/recruit-communications/pyqubo)
* [マニュアル](https://pyqubo.readthedocs.io/en/latest/)

# 3: まとめ・開発するときに覚えておくとよいこと
今回利用したタブー探索(QBSolv), 焼きなまし法(SimulatedAnnealingSampler), 量子アニーリング(DWaveSampler)は全て、`dimod.sampleABCMeta`という抽象クラスを継承して作成されているので、`sample_qubo`や`sample_ising`、`sample`といったメソッドで処理を呼び出し、`SampleSet`クラスによって結果を返すというふうに規格が統一されている(結果を集計してくれるかどうかや、num_reads等のオプションを受け付けてくれるか等の細かい仕様はことなるが)。自分でソルバーを開発する場合も、同じように`dimod.sampleABCMeta`を継承しつつ`SimulatedAnnealingSampler`のソースコードなどを参考に進めていくとよい。

ここでは紹介できなかったが、種々の実用的な組合せ最適化問題のQUBOによる定式化は https://qard.is.tohoku.ac.jp/T-Wave/?page_id=603 に詳しく解説されている。

またQUBOとイジングモデルと統一して`BQM`クラスとして問題を扱えること、制約条件を表すQUBOを重み付きで問題に足し合わせることも実演した。